In [1]:
from ox_db.oxd import OxDoc

In [2]:
doc = OxDoc("note.oxd")
doc.doc, doc.doc_path

('note', 'note.oxd')

In [30]:
doc.set("k1", " dummy data-1")
doc.set("k2", [" dummy data-1"])
doc.set("k3", {"data": " dummy data-3"})
doc.set("null-data", {"data": "lakajsjnxubebbdiwdiwi"})

True

In [31]:
doc.set("k2", 2)
doc.delete("null-data")

True

In [32]:
new_data = {"key4": "value4", "key5": "value5"}
doc.add(new_data)

In [33]:
doc.get("k1")
doc.get("null-data")
doc.get("key5")

ox-db :  dummy data-1
ox-db : value5


'value5'

In [34]:
doc.load_data()

{'oxd-init': 'data',
 '33': '55555555555555555555555',
 'key2': {'nested': 'value2'},
 'dd': 'dddd',
 'key4': 'value4',
 'key5': 'value5',
 'k1': ' dummy data-1',
 'k3': {'data': ' dummy data-3'},
 'k2': 2}

In [35]:
doc.compact()

{'oxd-init': 'data',
 '33': '55555555555555555555555',
 'key2': {'nested': 'value2'},
 'dd': 'dddd',
 'key4': 'value4',
 'key5': 'value5',
 'k1': ' dummy data-1',
 'k3': {'data': ' dummy data-3'},
 'k2': 2}

In [ ]:
doc.to_json()

In [5]:
doc.zip()

'note.oxd.zip'

In [9]:
OxDoc.unzip("./note.oxd.zip", ".")

'./note.oxd'

In [27]:
type("ss")

str

In [2]:
import asyncio

In [25]:
async def fetch_data(url):
    # Simulates making an API call
    print(f"t0  = {url}")
    await asyncio.sleep(10)  # Replace with actual network call
    print(f"t1  = {url}")
    return f"Data from {url}"


async def main():
    tasks = [
        fetch_data("dd1"),
        fetch_data("dd2"),
    ]
    print("x1")
    results = await asyncio.gather(*tasks)  # Run tasks concurrently
    print("x2")
    for result in results:
        print(result)
    print("x3")


# Call main coroutine directly (assuming Jupyter notebook environment)
print("x0")
tasksp = [main(), main() ,main(),main(),main(),main(),main()]

await asyncio.gather(*tasksp)

print("x5")

x0
x1
x1
x1
x1
x1
x1
x1
t0  = dd1
t0  = dd2
t0  = dd1
t0  = dd2
t0  = dd1
t0  = dd2
t0  = dd1
t0  = dd2
t0  = dd1
t0  = dd2
t0  = dd1
t0  = dd2
t0  = dd1
t0  = dd2
t1  = dd1
t1  = dd2
t1  = dd1
t1  = dd2
t1  = dd1
t1  = dd2
t1  = dd1
t1  = dd2
t1  = dd1
t1  = dd2
t1  = dd1
t1  = dd2
t1  = dd1
t1  = dd2
x2
Data from dd1
Data from dd2
x3
x2
Data from dd1
Data from dd2
x3
x2
Data from dd1
Data from dd2
x3
x2
Data from dd1
Data from dd2
x3
x2
Data from dd1
Data from dd2
x3
x2
Data from dd1
Data from dd2
x3
x2
Data from dd1
Data from dd2
x3
x5
